In [11]:
import numpy as np
import tensorflow as tf
sess = tf.Session()

x = tf.constant([[1,1,1],#4*3
                [1,1,1],
                [1,1,1],
                [1,1,1]], dtype=tf.float32)
c_ = 0.1 * tf.constant([[1,0,0,0],#4*4
                        [0,1,0,0],
                        [0,0,1,0],
                        [0,0,0,1]], dtype=tf.float32)
h_ = 0.1 * tf.constant([[1,1,1,1],#4*4
                       [1,1,1,1],
                       [1,1,1,1],
                       [1,1,1,1]], dtype=tf.float32)

num_units = len(sess.run(c_))
class lstm:
    def build(c, h):
        args = tf.concat((x,h), axis=1)
        print(args)

        out_size = 4 * num_units
        proj_size = args.shape[-1]
        print(out_size)
        print(proj_size)

        weights = tf.ones([proj_size, out_size]) * 0.5
        print(weights)


        out = tf.matmul(args, weights)
        print(out)

        bias = tf.ones([out_size]) * 0.5
        print(bias)

        concat = out + bias
        print(concat)

        i, j, f, o = tf.split(concat, 4, 1)
        print(i)
        print(j)
        print(f)
        print(o)

        g = tf.tanh(j)
        print(g)

        def sigmoid_array(x):
          return 1 / (1 + tf.exp(-x))

        forget_bias = 1.0

        sigmoid_f = sigmoid_array(f + forget_bias)
        print(sigmoid_f)

        sigmoid_array(i) * g

        new_c = c * sigmoid_f + sigmoid_array(i) * g
        print(new_c)

        new_h = tf.tanh(new_c) * sigmoid_array(o)
        print(new_h)

        print('\n new_h:',new_h)
        print('\n new_c',new_c)

#        print(res[1].h)
#        print(res[1].c)

        return new_c, new_h

In [12]:
#ta = tf.TensorArray(size=1, dtype=tf.float32)
ta_c = tf.TensorArray(size=100, dtype=tf.float32)
ta_h = tf.TensorArray(size=100, dtype=tf.float32)

In [13]:
def body(last_state, last_output, step, ta_c, ta_h):
    
    output = lstm.build(last_state, last_output)[0]
    state = lstm.build(last_state, last_output)[1]
    ta_c = ta_c.write(step, state)
    ta_h = ta_h.write(step, output)
    return state, output, tf.add(step, 1), ta_c, ta_h
    

timesteps = 100

steps = lambda a, b, step, c, d: tf.less(step, timesteps)

lstm_output, lstm_state, step, ta_c, ta_h = tf.while_loop(steps, body, (c_, h_, 0, ta_c, ta_h), parallel_iterations=20)


Tensor("while_2/concat:0", shape=(4, 7), dtype=float32)
16
7
Tensor("while_2/mul:0", shape=(7, 16), dtype=float32)
Tensor("while_2/MatMul:0", shape=(4, 16), dtype=float32)
Tensor("while_2/mul_1:0", shape=(16,), dtype=float32)
Tensor("while_2/add:0", shape=(4, 16), dtype=float32)
Tensor("while_2/split:0", shape=(4, 4), dtype=float32)
Tensor("while_2/split:1", shape=(4, 4), dtype=float32)
Tensor("while_2/split:2", shape=(4, 4), dtype=float32)
Tensor("while_2/split:3", shape=(4, 4), dtype=float32)
Tensor("while_2/Tanh:0", shape=(4, 4), dtype=float32)
Tensor("while_2/truediv:0", shape=(4, 4), dtype=float32)
Tensor("while_2/add_5:0", shape=(4, 4), dtype=float32)
Tensor("while_2/mul_5:0", shape=(4, 4), dtype=float32)

 new_h: Tensor("while_2/mul_5:0", shape=(4, 4), dtype=float32)

 new_c Tensor("while_2/add_5:0", shape=(4, 4), dtype=float32)
Tensor("while_2/concat_1:0", shape=(4, 7), dtype=float32)
16
7
Tensor("while_2/mul_6:0", shape=(7, 16), dtype=float32)
Tensor("while_2/MatMul_1:0", shap

In [14]:
print(sess.run(ta_c.read(99)))

[[0.95805687 0.95805687 0.95805687 0.95805687]
 [0.95805687 0.95805687 0.95805687 0.95805687]
 [0.95805687 0.95805687 0.95805687 0.95805687]
 [0.95805687 0.95805687 0.95805687 0.95805687]]


In [15]:
print(sess.run(ta_h.read(95)))

[[1.9543779 1.9543779 1.9543779 1.9543779]
 [1.9543779 1.9543779 1.9543779 1.9543779]
 [1.9543779 1.9543779 1.9543779 1.9543779]
 [1.9543779 1.9543779 1.9543779 1.9543779]]


In [16]:
print(sess.run(ta_h.read(99)))

[[1.9543779 1.9543779 1.9543779 1.9543779]
 [1.9543779 1.9543779 1.9543779 1.9543779]
 [1.9543779 1.9543779 1.9543779 1.9543779]
 [1.9543779 1.9543779 1.9543779 1.9543779]]
